# Challenge SD207 - 2017
*<p>Author: Pengfei MI, Rui SONG</p>*
*<p>Date: 06/06/2017</p>*

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from presets import Preset
from time import time

In [ ]:
# Define some usefull functions
def load_sound_files(file_paths):
    raw_sounds = []
    for fp in file_paths:
        X,sr = librosa.load(fp)
        raw_sounds.append(X)
    return raw_sounds

In [12]:
# Read data and preprocessing
X_test = pd.read_csv('data/test_files.txt', header=None)[0].values
#print X_test
data = pd.read_csv('data/audio/train.txt', header=None, sep='\s+')
X_train =  data[0].values
y_train = data[1].values
#print X_train
#print y_train